In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import pickle
from scipy.optimize import shgo
from mpl_toolkits.mplot3d import Axes3D
import time
import datetime

import topo_generic as topo
import JHD
import importlib
importlib.reload(topo)
importlib.reload(JHD)

#replace with %matplotlib inline if not using jupyter lab / widget
%matplotlib widget 

# Fig5 computation

**Algorithm summary**

The goal of this algorithm is to find the minimum excitation in the $\phi_L,\phi_R$ plane, for different values of $\phi_B$ and $n_g$.

To do so, I first fix $\phi_B$. Then I look for the minimum excitation energy in the whole $\phi_L,\phi_R$ plane, and in a small $n_g$ interval.

Due to the symmetry of the circuit, I only compute the energy gap for $n_g$ in [0,0.5], and for $\phi_B$ in [0,$pi$]. Then I just mirror the data for the whole parameter space.

**Runtime**

This computation requires quite some time for a high resolution as in figure 5 of https://arxiv.org/pdf/2009.03291.pdf 

On a regular desktop computer, it may take more than a day.

It helps to parallelize the computation.

The datas used in https://arxiv.org/pdf/2009.03291.pdf are loaded in part 2.



In [6]:
%%time

Ej=1
Ec=1

N=13

asym=[1,0.8,1.1,0.9,1.2]

tol=1e-8

phiBlist=np.linspace(0,np.pi,101)
dimphiB=len(phiBlist)

nglist=np.linspace(0,0.5,101)
dimng=len(nglist)
    
deg_gap=np.zeros([dimphiB,dimng])
deg_pos=np.zeros([dimphiB,dimng,4]) #4 coordinates: L, R, ng, phiB
deg_time=np.zeros([dimphiB,dimng])

for (iphiB,phiB) in enumerate(phiBlist):
    print('phiB : ',phiB)
    print('{datetime}'.format(datetime=datetime.datetime.now()))
    param=[N,Ej,Ec,phiB,asym,tol]
    start_time=time.time()
    
    for (ing,ng) in enumerate(nglist):
        start_time_ng=time.time()
        print(ing)
        if ing%10==0:
            print(f'ing is {ing}')
        if ing==0: continue ##skip first iteration to use end of interval to define it
            
        bnds=((0,2*np.pi), (0,2*np.pi), (nglist[ing-1],nglist[ing]))
        
        temp_deg=shgo(JHD.minimize_phiL_phiR_ng,bnds,args=param, \
                      n=151, iters=15, sampling_method='sobol', \
                      minimizer_kwargs={'method':'SLSQP', 'options':{'options':{'ftol':1e-8, 'eps':1e-10, 'maxiter':100}}}, \
                      options={'ftol':1e-8, 'maxtime':60, 'local_iter':10} )


        deg_time[iphiB,ing]=time.time()-start_time
        deg_gap[iphiB,ing]=temp_deg.fun
        deg_pos[iphiB,ing]=np.hstack([temp_deg.x,phiB])
        
        print(f'gap: {deg_gap[iphiB,ing]} \t ng: {deg_pos[iphiB,ing,2]} \t phiB: {deg_pos[iphiB,ing,3]}')
        if ing%10==0:
            print(f'this ng run took {time.time()-start_time_ng:.4f}sec')
        print()
    print(f'{time.time()-start_time}sec for this phiB run')
    print(f'estimate remaining time to {(time.time()-start_time)*(dimphiB-iphiB+1)/60}min')

phiB :  0.031415926535897934
2020-09-14 17:55:31.868232
0
ing is 0
1


KeyboardInterrupt: 

# load data and display fig5

**Load data**

In [19]:
phiBlist=np.linspace(0,np.pi,101)
nglist=np.linspace(0,0.5,101)

with open('fig5_data/JHD_fig5.pkl', 'rb') as outfile:
    deg_gap,deg_pos=pickle.load(outfile)

**mirror symmetry to enlarge to full parameter space**

In [20]:
def unfold(phiBlist,nglist,deg_gap):
    phiBlist=np.linspace(-np.pi,np.pi,2*len(phiBlist)-1)
    
    #nglist: remove last element, then shift everything by half step (so now elem i corresponds to middle of interval i)
    nglist=np.linspace(0,1,2*len(nglist)-1)[:-1]
    nglist=nglist+0.5*(nglist[1]-nglist[0])
    
    ## remove phiB=pi from second set
    deg_gap=np.concatenate((deg_gap[::-1][1:],deg_gap),0)
    #unfold on ng: remove first ng point, unfold, then remove first elem which is centered at ng=0.5, counted twice
    deg_gap=np.concatenate((deg_gap,deg_gap[:,1:][:,::-1][:,1:]),1)

    return phiBlist,nglist,deg_gap

In [21]:
phiBlist_, nglist_, deg_gap_=unfold(phiBlist,nglist,deg_gap)

In [29]:
plt.close('all')
fig=plt.figure(figsize=(12,8))
plt.rcParams.update({'font.size': 14})

X,Y = np.meshgrid(phiBlist_,nglist_,indexing='ij')  ### NEED indexing='ij' SO THAT MESHGRID DOES NOT INVERT X AND Y

plt.pcolormesh(X,Y,deg_gap_[:,1:]/np.sqrt(2), shading='nearest', norm=colors.LogNorm(vmin=1e-3), cmap='viridis') 
plt.colorbar(label='Energy gap (units of $\hbar \omega_p$)')
plt.xlabel('$\phi_B$')
plt.ylabel('$n_g$')
plt.xticks(np.arange(-np.pi,1.5*np.pi,0.5*np.pi))
plt.yticks(np.arange(0.25,1,0.25))

plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …